## Geocoding and Visualizing Centuries scored by Sachin Tendulkar

Sachin Tendulkar is one of the best batsmen of history of cricket. Sachin hold the world record for scoring highest number of centuries in international cricket playing formats. This exercise consists of a section of geocoding the venues where Sachin  scored his centuties and a visualzation of basic statistics. 

The data for this exercise is imported from the Wikipedia page [List of international cricket centuries by Sachin Tendulkar](https://en.wikipedia.org/wiki/List_of_international_cricket_centuries_by_Sachin_Tendulkar).

In [18]:
# Import Libraries
import pandas as pd
from geopandas.tools import geocode
from geopy.geocoders import Nominatim
import time
from pprint import pprint
import folium
from folium import plugins
from folium.plugins import HeatMap
import requests

In [2]:
# Import data from the wikipedia page
data=pd.read_html('https://en.wikipedia.org/wiki/List_of_international_cricket_centuries_by_Sachin_Tendulkar')

The Wikipedia page consists of multiple tables that include different sets of information on Sachin's centuries.

In [3]:
# Number of tables in the Wikipedia page
len(data)

6

The first table defines the column names of the tables.

In [4]:
# column names of the tables in the Wikipedia page
data[0]

,Symbol,Meaning
0,*,Remained not out
1,NaN,Player of the match
2,NaN,Captained the Indian cricket team
3,Test,The number of the Test match played in that se...
4,Pos.,Position in the batting order
5,Inn.,The innings of the match
6,S.R.,Strike rate during the innings
7,H/A/N,"Venue was at home (India), away or neutral"
8,Lost,The match was lost by India
9,Won,The match was won by India


Second and third tables include the information of Test and ODI (One Day International) centuries. The tables consist of information such as venue, the opponent, and date etc.

It would be easy to analyze all the data if we join these two tables. For the ease of identification a new column named format is added and it will indicate whether the cricket format of the row is Test or ODI>

In [5]:
# Table with information on Test centuries
centuries_test = data[1]
centuries_test['Format'] = 'Test'
centuries_test.head(5)

,No.,Score,Against,Pos.,Inn.,Test,Venue,H/A,Date,Result,Format
0,1,119*,England,6,4,2,"Old Trafford, Manchester",Away,9 August 1990,Drawn[11],Test
1,2,148*,Australia,6,2,3,"Sydney Cricket Ground, Sydney",Away,2 January 1992,Drawn[12],Test
2,3,114,Australia,4,2,5,"WACA Ground, Perth",Away,1 February 1992,Lost[13],Test
3,4,111,South Africa,4,2,2,"Wanderers Stadium, Johannesburg",Away,26 November 1992,Drawn[14],Test
4,5,165,England,4,1,2,"M. A. Chidambaram Stadium, Chennai",Home,11 February 1993,Won[15],Test


In [6]:
# Table with information on ODI centuries
centuries_odi = data[2]
centuries_odi['Format'] = 'ODI'
centuries_odi.head(5)

,No.,Score,Against,Pos.,Inn.,S/R,Venue,H/A/N,Date,Result,Format
0,1,110,Australia,2,1,84.61,"R. Premadasa Stadium, Colombo",Neutral,9 September 1994,Won[63],ODI
1,2,115,New Zealand,2,2,84.55,"IPCL Sports Complex Ground, Vadodara",Home,28 October 1994,Won[64],ODI
2,3,105,West Indies,2,1,78.35,"Sawai Mansingh Stadium, Jaipur",Home,11 November 1994,Won[65],ODI
3,4,112*,Sri Lanka,2,2,104.67,"Sharjah Cricket Association Stadium, Sharjah",Neutral,9 April 1995,Won[66],ODI
4,5,127*,Kenya,2,2,92.02,"Barabati Stadium, Cuttack",Home,18 February 1996,Won[67],ODI


Lets combine the tables row-wise.   

In [7]:
# Concatenate two tables into one table
all_centuries = pd.concat([centuries_test,centuries_odi])

As the next step, name of the city where Sachin scored his each century is extracted from the venue column. The extracted city name is used to obtain the longitude and latitude values of the city. Geographic information is acquired from the open source service called [Photon](https://photon.komoot.io/). 

In [15]:
# Initialize three empty lists to add name and geographic coordinates of each city
city_name = []
longitude = []
latitude = []

for index, row in all_centuries.iterrows():
    try:
        ground = row['Venue']
        # Split the venue name and extract and print the name of the city
        city = ground.split(", ")[-1]
        print(city)
        # Obtain the geographic information of the city 
        geographic_info= geocode(city, provider='photon')
        # Convert the geographic information to string, extract coordinates, and append the coordinates to the lists 
        location = str(geographic_info['geometry'][0])
        location_coordinates = location.split(" ")
        city_name.append(city)
        longitude.append(location_coordinates[1][1:])
        latitude.append(location_coordinates[2][:-1])
    except requests.exceptions.Timeout:
        print("Timeout occurred")
    
    except TypeError:
            print('geocoding information for '+ row['Venue'] + ' is not found')

Manchester
Sydney
Perth
Johannesburg
Chennai
Colombo
Lucknow
Nagpur
Birmingham
Nottingham
Cape Town
Colombo
Colombo
Mumbai
Chennai
Bangalore
Wellington
Chennai
Colombo
Mohali
Ahmedabad
Melbourne
New Delhi
Nagpur
Chennai
Bloemfontein
Ahmedabad
Nagpur
Port of Spain
Leeds
Kolkata
Sydney
Multan
Dhaka
New Delhi
Chittagong
Mirpur
Sydney
Adelaide
Nagpur
Chennai
Hamilton
Ahmedabad
Chittagong
Mirpur
Nagpur
Kolkata
Colombo
Bangalore
Centurion
Cape Town
Colombo
Vadodara
Jaipur
Sharjah
Cuttack
New Delhi
Singapore
Sharjah
Colombo
Mumbai
Benoni
Bangalore
Kanpur
Sharjah
Sharjah
Kolkata
Colombo
Bulawayo
Dhaka
Sharjah
Sharjah
Bristol
Colombo
Hyderabad
Vadodara
Sharjah
Jodhpur
Indore
Harare
Johannesburg
Paarl
Chester-le-Street
Bristol
Pietermaritzburg
Gwalior
Hyderabad
Rawalpindi
Ahmedabad
Peshawar
Kuala Lumpur
Vadodara
Sydney
Christchurch
Colombo
Hyderabad
Gwalior
Bangalore
Nagpur
Mirpur


In [16]:
# Add the city name and the location coordinates as new columns to the table
all_centuries["Longitude"] = longitude
all_centuries["Latitude"] = latitude
all_centuries["City"] = city_name
all_centuries.head(5)

,No.,Score,Against,Pos.,Inn.,Test,Venue,H/A,Date,Result,Format,S/R,H/A/N,Longitude,Latitude,City
0,1,119*,England,6,4,2.0,"Old Trafford, Manchester",Away,9 August 1990,Drawn[11],Test,NaN,NaN,-2.2451148,53.4794892,Manchester
1,2,148*,Australia,6,2,3.0,"Sydney Cricket Ground, Sydney",Away,2 January 1992,Drawn[12],Test,NaN,NaN,150.9568559523945,-33.768528,Sydney
2,3,114,Australia,4,2,5.0,"WACA Ground, Perth",Away,1 February 1992,Lost[13],Test,NaN,NaN,115.8604796,-31.9527121,Perth
3,4,111,South Africa,4,2,2.0,"Wanderers Stadium, Johannesburg",Away,26 November 1992,Drawn[14],Test,NaN,NaN,28.049722,-26.205,Johannesburg
4,5,165,England,4,1,2.0,"M. A. Chidambaram Stadium, Chennai",Home,11 February 1993,Won[15],Test,NaN,NaN,80.3005235,13.1116462,Chennai


Now the Longitude and Latitude information are acquired. These locations can be plotted in a world map using folium. A marker is added to indicate each city. When you click on the marker, the city name and the format which the century is scored will appear.

In [19]:
# Open a worldmap using Openstreetmap
m= folium.Map(tiles='Openstreetmap', zoom_start=2)

# Add a marker to each point and set the information to appear when click on the marker
for index,row in all_centuries.iterrows():
    folium.Marker([row['Latitude'],row['Longitude']], popup= 'City: '+ row['City'] 
                  + '<br>''<br>''Format: ' +row['Format']).add_to(m)

m

Instantialte a client instance to connect the geodoing service.

In [24]:
# instantiate a new Nominatim client
app = Nominatim(user_agent="geocoding")

Define a function to get the address of each location indicated by each pair of longitudes and latitudes.

In [25]:
def get_address_by_location(latitude, longitude, language="en"):
    # build coordinates string to pass to the reverse function
    coordinates = f"{latitude}, {longitude}"
    # sleep for a second to respect the geocoding service usage Policy
    time.sleep(1)
    try:
        return app.reverse(coordinates, language=language).raw
    except:
        return get_address_by_location(latitude, longitude)

In [22]:
# Initialize an empty list to add country names
country= []

for rowIndex, row in all_centuries.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    # get the geographic information of each location
    address = get_address_by_location(latitude, longitude)
    
    # Add the name of the country correspond to each entry
    country.append(address['address']['country'])
print(country)

['United Kingdom', 'Australia', 'Australia', 'South Africa', 'India', 'Sri Lanka', 'India', 'India', 'United Kingdom', 'United Kingdom', 'South Africa', 'Sri Lanka', 'Sri Lanka', 'India', 'India', 'India', 'New Zealand', 'India', 'Sri Lanka', 'India', 'India', 'Australia', 'India', 'India', 'India', 'South Africa', 'India', 'India', 'Trinidad and Tobago', 'United Kingdom', 'India', 'Australia', 'Pakistan', 'Bangladesh', 'India', 'Bangladesh', 'Pakistan', 'Australia', 'Australia', 'India', 'India', 'Canada', 'India', 'Bangladesh', 'Pakistan', 'India', 'India', 'Sri Lanka', 'India', 'South Africa', 'South Africa', 'Sri Lanka', 'India', 'India', 'United Arab Emirates', 'India', 'India', 'Singapore', 'United Arab Emirates', 'Sri Lanka', 'India', 'South Africa', 'India', 'India', 'United Arab Emirates', 'United Arab Emirates', 'India', 'Sri Lanka', 'Zimbabwe', 'Bangladesh', 'United Arab Emirates', 'United Arab Emirates', 'United Kingdom', 'Sri Lanka', 'India', 'India', 'United Arab Emirates

In [23]:
# Add the name of the country as a new columns to the table
all_centuries["Country"] = country
all_centuries.head(5)

,No.,Score,Against,Pos.,Inn.,Test,Venue,H/A,Date,Result,Format,S/R,H/A/N,Longitude,Latitude,City,Country
0,1,119*,England,6,4,2.0,"Old Trafford, Manchester",Away,9 August 1990,Drawn[11],Test,NaN,NaN,-2.2451148,53.4794892,Manchester,United Kingdom
1,2,148*,Australia,6,2,3.0,"Sydney Cricket Ground, Sydney",Away,2 January 1992,Drawn[12],Test,NaN,NaN,150.9568559523945,-33.768528,Sydney,Australia
2,3,114,Australia,4,2,5.0,"WACA Ground, Perth",Away,1 February 1992,Lost[13],Test,NaN,NaN,115.8604796,-31.9527121,Perth,Australia
3,4,111,South Africa,4,2,2.0,"Wanderers Stadium, Johannesburg",Away,26 November 1992,Drawn[14],Test,NaN,NaN,28.049722,-26.205,Johannesburg,South Africa
4,5,165,England,4,1,2.0,"M. A. Chidambaram Stadium, Chennai",Home,11 February 1993,Won[15],Test,NaN,NaN,80.3005235,13.1116462,Chennai,India


Now the dataset is completed with a lot of usable location and information. This data can be used for further analysis and visualisations.

In [26]:
# # add counts
# df= all_centuries.sort_values(['Country'])
# filter = df["Format"]=="Test"
# df.where(filter)
# df

In [27]:
# data = []
# # To create a heatmap, group the 
# for index,row in all_centuries.iterrows():
#     location_data
#     for i in range(len(all_centuries)):
        
    
# HeatMap().add_to(m)    

# m